In [5]:
import os, pickle
import numpy as np

from utils import pickle_load
from collections import Counter

ImportError: attempted relative import with no known parent package

In [ ]:
data_dir = './remi_dataset'
polyph_out_dir = './remi_dataset/attr_cls/polyph'
rhythm_out_dir = './remi_dataset/attr_cls/rhythm'

rhym_intensity_bounds = [0.2, 0.25, 0.32, 0.38, 0.44, 0.5, 0.63]
polyphonicity_bounds = [2.63, 3.06, 3.50, 4.00, 4.63, 5.44, 6.44]

def compute_polyphonicity(events, n_bars):
  poly_record = np.zeros( (n_bars * 16,) )

  cur_bar, cur_pos = -1, -1
  for ev in events:
    if ev['name'] == 'Bar':
      cur_bar += 1
    elif ev['name'] == 'Beat':
      cur_pos = int(ev['value'])
    elif ev['name'] == 'Note_Duration':
      duration = int(ev['value']) // 120
      st = cur_bar * 16 + cur_pos
      poly_record[st:st + duration] += 1
  
  return poly_record

def get_onsets_timing(events, n_bars):
  onset_record = np.zeros( (n_bars * 16,) )

  cur_bar, cur_pos = -1, -1
  for ev in events:
    if ev['name'] == 'Bar':
      cur_bar += 1
    elif ev['name'] == 'Beat':
      cur_pos = int(ev['value'])
    elif ev['name'] == 'Note_Pitch':
      rec_idx = cur_bar * 16 + cur_pos
      onset_record[ rec_idx ] = 1

  return onset_record

In [ ]:
pieces = [p for p in sorted(os.listdir(data_dir)) if '.pkl' in p]
print(pieces)
all_r_cls = []
all_p_cls = []

if not os.path.exists(polyph_out_dir):
    os.makedirs(polyph_out_dir)
if not os.path.exists(rhythm_out_dir):
    os.makedirs(rhythm_out_dir)  

for p in pieces:
    bar_pos, events = pickle_load(os.path.join(data_dir, p))
    events = events[ :bar_pos[-1] ]

    polyph_raw = np.reshape(
            compute_polyphonicity(events, n_bars=len(bar_pos)), (-1, 16)
        )
    rhythm_raw = np.reshape(
            get_onsets_timing(events, n_bars=len(bar_pos)), (-1, 16)
        )

    polyph_cls = np.searchsorted(polyphonicity_bounds, np.mean(polyph_raw, axis=-1)).tolist()
    rfreq_cls = np.searchsorted(rhym_intensity_bounds, np.mean(rhythm_raw, axis=-1)).tolist()

    print('polyph_cls', polyph_cls)
    print('rfreq_cls', rfreq_cls)

    pickle.dump(polyph_cls, open(os.path.join(
            polyph_out_dir, p), 'wb'
        ))
    pickle.dump(rfreq_cls, open(os.path.join(
            rhythm_out_dir, p), 'wb'
        ))

    all_r_cls.extend(rfreq_cls)
    all_p_cls.extend(polyph_cls)

print ('[rhythm classes]', Counter(all_r_cls))
print ('[polyph classes]', Counter(all_p_cls))

In [1]:
import os

def splitting_dataset(src, train_size=0.8, val_size=0.5):
  data_list = [os.path.join(src, f) for f in os.listdir(src) if os.path.isfile(os.path.join(src, f))]

  train_pieces = int(len(data_list)*train_size)
  train_samples = data_list[: train_pieces]
  val_samples = data_list[train_pieces: ]

  val_pieces = int(len(val_samples)*val_size)
  val_samples = val_samples[: val_pieces]
  test_samples = val_samples[val_pieces: ]
  return train_samples, val_samples, test_samples


train_samples, val_samples, test_samples = splitting_dataset(src=r'D:\UNI\Thesis\NeuralNotes\data\resource', train_size=0.8, val_size=0.5)

In [2]:
from dataloader import REMIFullSongTransformerDataset

data_dir = r'D:\UNI\Thesis\NeuralNotes\data\resource'
vocab_path = r'D:\UNI\Thesis\NeuralNotes\agent\pickles\remi_vocab_v2.pkl'

print('Dataset: Training Samples')
dset = REMIFullSongTransformerDataset(
  data_dir, vocab_path,
  do_augment=True,
  model_enc_seqlen=128,
  model_dec_seqlen=1280,
  model_max_bars=16,
  pieces=train_samples,
  pad_to_same=True
)

Dataset: Training Samples
[preparing data] now at #0


In [3]:
from model_factory import Factory


factory = Factory()
model, config = factory.generate(dset=dset)

config

{'data': {'data_dir': './remi_dataset',
  'train_split': './pickles/train_pieces.pkl',
  'val_split': './pickles/val_pieces.pkl',
  'test_split': './pickles/test_pieces.pkl',
  'vocab_path': './pickles/remi_vocab_v2.pkl',
  'max_bars': 16,
  'enc_seqlen': 128,
  'dec_seqlen': 1280,
  'batch_size': 4},
 'model': {'enc_n_layer': 6,
  'enc_n_head': 4,
  'enc_d_model': 256,
  'enc_d_ff': 1024,
  'dec_n_layer': 6,
  'dec_n_head': 4,
  'dec_d_model': 256,
  'dec_d_ff': 1024,
  'd_embed': 256,
  'd_latent': 64,
  'd_polyph_emb': 32,
  'd_rfreq_emb': 32,
  'cond_mode': 'in-attn',
  'pretrained_params_path': None,
  'pretrained_optim_path': None},
 'training': {'device': 'cuda:0',
  'ckpt_dir': './ckpt/mm_small.pt',
  'trained_steps': 0,
  'max_epochs': 1000,
  'max_lr': 0.0001,
  'min_lr': 5e-06,
  'lr_warmup_steps': 200,
  'lr_decay_steps': 150000,
  'no_kl_steps': 10000,
  'kl_cycle_steps': 5000,
  'kl_max_beta': 1.0,
  'free_bit_lambda': 0.25,
  'constant_kl': False,
  'ckpt_interval': 50,
